# **Facial Expression Classificationt**

In [ ]:
# Facial Expression Classification dataset
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import Module, Conv2d, Linear, MaxPool2d, Dropout, ReLU, LogSoftmax
from torch import flatten
import matplotlib
matplotlib.use("Agg")
from sklearn.metrics import classification_report
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch import nn
from sklearn.model_selection import KFold
from sklearn.preprocessing import Normalizer, normalize
import matplotlib
matplotlib.use('Agg')


train_data = 'train_data.csv'
train_labels = 'train_target.csv'
test_data = 'test_data.csv'

# Read the CSV file into a pandas DataFrame
train_data = pd.read_csv(train_data, header=None)
train_labels = pd.read_csv(train_labels, header=None)
test_data = pd.read_csv(test_data, header=None)

# display the top rows
train_data.head(10)


In [14]:
# Definee Training Data
train_data_np = train_data.to_numpy()
test_data_np = test_data.to_numpy()
train_data_np = np.float32(train_data_np)
train_labels_np = train_labels.to_numpy()

sample_image = np.reshape(train_data_np[0,:], (48,48))

plt.imshow(sample_image)
plt.show()

C:\Users\kelz9\AppData\Local\Temp\ipykernel_29592\2912847410.py:10: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [15]:
# Reset model weights to avoid weight leakage
def reset_weights(m):
  for layer in m.children():
   if hasattr(layer, 'reset_parameters'):
    print(f'Reset trainable parameters of layer = {layer}')
    layer.reset_parameters()

In [52]:
# Define Convolutional Neural Network
class CNN(Module):
	def __init__(self, numChannels, classes):
		# call the parent constructor
		super(CNN, self).__init__()
		# initialize first set of CONV => RELU => POOL layers
		self.conv1 = Conv2d(in_channels=numChannels, out_channels=20, kernel_size=(5, 5))
		self.relu1 = ReLU()
		self.dropout1 = Dropout(0.2)
		self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
		# initialize second set of CONV => RELU => POOL layers
		self.conv2 = Conv2d(in_channels=20, out_channels=30, kernel_size=(5, 5))
		self.relu2 = ReLU()
		self.dropout2 = Dropout(0.2)
		self.maxpool2 = MaxPool2d(kernel_size=(2, 2), stride=(1, 1))
		# initialize second set of CONV => RELU => POOL layers
		self.conv3 = Conv2d(in_channels=30, out_channels=40, kernel_size=(5, 5))
		self.relu3= ReLU()
		self.dropout3 = Dropout(0.2)
		self.maxpool3 = MaxPool2d(kernel_size=(2, 2), stride=(1, 1))
		# initialize first (and only) set of FC => RELU layers
		self.fc1 = Linear(in_features=5760, out_features=1000)
		self.relu4 = ReLU()
		self.dropout4 = Dropout(0.2)
		# initialize our softmax classifier
		self.fc2 = Linear(in_features=1000, out_features=classes)
		self.logSoftmax = LogSoftmax(dim=1)
		
	# Define forward pass function
	def forward(self, x):
		# pass the input through our first set of CONV => RELU =>
		# POOL layers
		x = self.conv1(x)
		x = self.relu1(x)
		#x = self.dropout1(x)
		x = self.maxpool1(x)
		# pass the output from the previous layer through the second
		# set of CONV => RELU => POOL layers
		x = self.conv2(x)
		x = self.relu2(x)
		#x = self.dropout2(x)
		x = self.maxpool2(x)
		# pass the output from the previous layer through the second
		# set of CONV => RELU => POOL layers
		x = self.conv3(x)
		x = self.relu3(x)
		#x = self.dropout3(x)
		x = self.maxpool3(x)
		# flatten the output from the previous layer and pass it
		# through our only set of FC => RELU layers
		x = flatten(x, 1)
		x = self.fc1(x)
		x = self.relu4(x)
		#x = self.dropout4(x)
		# pass the output to our softmax classifier to get our output
		# predictions
		x = self.fc2(x)
		output = self.logSoftmax(x)
		# return the output predictions
		return output

In [47]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class AngularPenaltySMLoss(nn.Module):

    def __init__(self, in_features, out_features, loss_type='arcface', eps=1e-7, s=None, m=None):
        super(AngularPenaltySMLoss, self).__init__()
        loss_type = loss_type.lower()
        assert loss_type in  ['arcface', 'sphereface', 'cosface']
        if loss_type == 'arcface':
            self.s = 64.0 if not s else s
            self.m = 0.5 if not m else m
        if loss_type == 'sphereface':
            self.s = 64.0 if not s else s
            self.m = 1.35 if not m else m
        if loss_type == 'cosface':
            self.s = 30.0 if not s else s
            self.m = 0.4 if not m else m
        self.loss_type = loss_type
        self.in_features = in_features
        self.out_features = out_features
        self.fc = nn.Linear(in_features, out_features, bias=False)
        self.eps = eps
    
    def forward(self, x, labels):
        '''
        input shape (N, in_features)
        '''
        assert len(x) == len(labels)
        assert torch.min(labels) >= 0
        assert torch.max(labels)         
        for W in self.fc.parameters():
            W = F.normalize(W, p=2, dim=1)

        x = F.normalize(x, p=2, dim=1)

        wf = self.fc(x)
        if self.loss_type == 'cosface':
            numerator = self.s * (torch.diagonal(wf.transpose(0, 1)[labels]) - self.m)
        if self.loss_type == 'arcface':
            numerator = self.s * torch.cos(torch.acos(torch.clamp(torch.diagonal(wf.transpose(0, 1)[labels]), -1.+self.eps, 1-self.eps)) + self.m)
        if self.loss_type == 'sphereface':
            numerator = self.s * torch.cos(self.m * torch.acos(torch.clamp(torch.diagonal(wf.transpose(0, 1)[labels]), -1.+self.eps, 1-self.eps)))

        excl = torch.cat([torch.cat((wf[i, :y], wf[i, y+1:])).unsqueeze(0) for i, y in enumerate(labels)], dim=0)
        denominator = torch.exp(numerator) + torch.sum(torch.exp(self.s * excl), dim=1)
        L = numerator - torch.log(denominator)

        return -torch.mean(L)
      



In [28]:
class ConvAngularPen(nn.Module):
    def __init__(self, num_classes=3, loss_type='arcface'):
        super(ConvAngularPen, self).__init__()
        self.convlayers = CNN(1, 3)
        self.adms_loss = AngularPenaltySMLoss(3, num_classes, loss_type=loss_type)

    def forward(self, x, labels=None, embed=False):
        #x = self.convlayers(x)
        if embed:
            return x
        L = self.adms_loss(x, labels)

        return L

In [ ]:
# define training hyperparameters
INIT_LR = 1e-3
BATCH_SIZE = 64
EPOCHS = 10
KFOLDS = 10

# For fold results
results = {}

loss_function = nn.CrossEntropyLoss()

# Define the K-fold Cross Validator
kfold = KFold(n_splits=KFOLDS, shuffle=True)

dataAll = [(((train_data_np[i,:])).reshape(1, 48, 48), train_labels_np[i]) for i in range(train_data_np.shape[0])]

for fold, (train_ids, val_ids) in enumerate(kfold.split(dataAll)):
    print(fold)
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    val_subsampler = torch.utils.data.SubsetRandomSampler(val_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = torch.utils.data.DataLoader(dataAll, batch_size=BATCH_SIZE, sampler=train_subsampler)
    valloader = torch.utils.data.DataLoader(dataAll, batch_size=BATCH_SIZE, sampler=val_subsampler)

    # Init the neural network
    network = CNN(numChannels=1, classes=3)
    network.apply(reset_weights)
    
    # Initialize optimizer
    optimizer = torch.optim.Adam(network.parameters(), lr=1e-4, weight_decay=0.001)

    model = ConvAngularPen(loss_type="sphereface")
    # Run the training loop for defined number of epochs
    for epoch in range(0, EPOCHS):
        current_loss = 0
        print(epoch)
        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):
            # Get inputs
            inputs, targets = data
            targets = torch.flatten(targets)            
            # Zero the gradients
            optimizer.zero_grad()
            
            # Perform forward pass
            outputs = network(inputs)
            
            # Compute loss
            loss = loss_function(outputs, targets)
            
            # Perform backward pass
            loss.backward()
        
            # Perform optimization
            optimizer.step()
            
            # Print statistics
            current_loss += loss.item()
            if i % 500 == 499:
                print('Loss after mini-batch %5d: %.3f' %
                    (i + 1, current_loss / 500))
                current_loss = 0.0

    # Process is complete.
    print('Training process has finished.')

    # Print about validating
    print('Starting validating')

    # Evaluationfor this fold
    correct, total = 0, 0
    correctV, totalV = 0, 0

    with torch.no_grad():
        # Iterate over the test data and generate predictions
        for i, data in enumerate(trainloader, 0):
            # Get inputs
            inputs, targets = data
            targets = torch.flatten(targets)            

            # Generate outputs
            outputs = network(inputs)

            # Set total and correct
            predicted = torch.max(outputs.data, 1)[1]
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    # Print accuracy
    print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correct / total))
    print('--------------------------------')
    results[fold] = 100.0 * (correct / total)

    with torch.no_grad():
        # Iterate over the test data and generate predictions
        for i, data in enumerate(valloader, 0):
            # Get inputs
            inputs, targets = data
            targets = torch.flatten(targets)            

            # Generate outputs
            outputs = network(inputs)

            # Set total and correct
            predicted = torch.max(outputs.data, 1)[1]
            totalV += targets.size(0)
            correctV += (predicted == targets).sum().item()

    # Print accuracy
    print('Accuracy for fold %d: %d %%' % (fold, 100.0 * correctV / totalV))
    print('--------------------------------')
    results[fold] = 100.0 * (correctV / totalV)

# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
sum = 0.0
for key, value in results.items():
    print(f'Fold {key}: {value} %')
    sum += value
    print(f'Average: {sum/len(results.items())} %')

In [35]:
test_data1 = np.reshape(test_data_np, (test_data_np.shape[0], 1, 48, 48))
test_data1 = torch.Tensor(test_data1)
# Generate outputs
outputs = network(test_data1)

# Set total and correct
predicted = torch.max(outputs.data, 1)[1]

predicted

tensor([2, 0, 1,  ..., 1, 2, 2])

In [121]:
# Create final submission file
test_ids = [i for i in range(test_data1.size(0))]

final_results = predicted.int()

x_np = final_results.numpy()
x_df = pd.DataFrame(x_np)
x_df.to_csv('sample_subsmission.csv')
